In [ ]:
#this file computes the number of people using the metro between timestamps

In [3]:
import pandas as pd
from datetime import datetime,timedelta
mta_data = pd.read_pickle('mta_data.pickle')

In [31]:
#reset index and drop old indices to remove duplicate indexes and fix cols
mta_data = mta_data.reset_index().drop('index',axis=1)
mta_data.columns = [col.strip() for col in mta_data.columns]


# convert dates to datetime, create col; convert that to day of week, create col; convert that to weekday T/F, create col
mta_data['weekday'] = pd.to_datetime(mta_data.DATE,format='%m/%d/%Y')
mta_data['weekday_n'] = mta_data.weekday.dt.dayofweek
mta_data['weekday_tf'] = mta_data.weekday_n<5

#create columns for entries since last time
mta_data['roll_ent'] = mta_data.groupby(['weekday'])['ENTRIES'].diff(periods = 1)
mta_data['roll_ex'] = mta_data.groupby(['weekday'])['EXITS'].diff(periods = 1)
mta_data.fillna(0,inplace=True)
mta_data.roll_ent[mta_data.roll_ent<0]=0
mta_data.roll_ex[mta_data.roll_ex<0]=0

#find mean of data by station and whether weekday or not
mean_entry = mta_data.groupby(['STATION','weekday_tf']).mean()
#mean total traffic, then only that
mean_entry['through'] = mean_entry.ENTRIES + mean_entry.EXITS
mean_entry.drop(labels = ['ENTRIES','EXITS','weekday_n'],axis=1,inplace=True)

#shift table so that each station's through is in cols T and F, then subtract
mean_entry.reset_index()
piv_me=mean_entry.pivot_table(ind
                              ex='STATION',columns='weekday_tf')
weekdaymean = piv_me['through']['weekday_tf'==False]
weekendmean = piv_me['through']['weekday_tf'==True]
touristdiff = weekdaymean-weekendmean

SyntaxError: invalid syntax (<ipython-input-31-f84717d6410b>, line 27)

In [30]:
touristdiffs[touristdiffs>0]

NameError: name 'touristdiffs' is not defined